# Chatham Submission
Quickly coded for final, see README for assignment requirements provided

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO
import datetime as dt

In [3]:
def _read_ghcn_by_years(station_id: str, start_year: int, end_year: int) -> pd.DataFrame:
    """
    Read GHCN daily CSVs from S3 (noaa-ghcn-pds/csv/by_year/YYYY.csv),
    filter to one station, keep TMAX/TMIN, and return a tidy DataFrame.
    """
    frames = []
    usecols = ["ID", "DATE", "ELEMENT", "DATA_VALUE"] 
    for year in range(start_year, end_year + 1):
        s3_path = f"s3://noaa-ghcn-pds/csv/by_year/{year}.csv"
        try:
            df_y = pd.read_csv(
                s3_path,
                storage_options={"anon": True},
                usecols=usecols,
                dtype={"ID": "string", "DATE": "string", "ELEMENT": "string", "DATA_VALUE": "Int64"},
                low_memory=False,
            )
        except Exception:
            # year file might not exist or be temporarily unavailable
            continue

        # Filter to station and elements of interest
        df_y = df_y[df_y["ID"] == station_id]
        if df_y.empty:
            continue

        df_y = df_y[df_y["ELEMENT"].isin(["TMAX", "TMIN"])].copy()
        df_y["DATE"] = pd.to_datetime(df_y["DATE"], errors="coerce")
        df_y = df_y.dropna(subset=["DATE"])
        df_y["TEMP_C"] = df_y["DATA_VALUE"] / 10.0  # tenths degC -> degC
        df_y["DOY"] = df_y["DATE"].dt.dayofyear
        df_y["YEAR"] = df_y["DATE"].dt.year
        frames.append(df_y[["DATE", "YEAR", "DOY", "ELEMENT", "TEMP_C"]])

    if not frames:
        return pd.DataFrame(columns=["DATE", "YEAR", "DOY", "ELEMENT", "TEMP_C"])

    df = pd.concat(frames, ignore_index=True)
    return df

In [8]:
# Plant City station
station_id = "USC00087205"

# 1991–2020 daily data
df = _read_ghcn_by_years(station_id, 1991, 2020)
df.head()

KeyboardInterrupt: 

### Part 1 (25 points)

In [ ]:
# My data took over 40m to load before I gave up
df = pd.read_csv("USC00087205_nesbitt.csv", parse_dates=["DATE"])
df.head()

C:\Users\aliso\AppData\Local\Temp\ipykernel_2188\2807247140.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("USC00087205_nesbitt.csv", parse_dates=["DATE"])


,DATE,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
0,1892-09-01,USC00087205,TMAX,322,NaN,NaN,6,NaN
1,1892-09-02,USC00087205,TMAX,317,NaN,NaN,6,NaN
2,1892-09-03,USC00087205,TMAX,317,NaN,NaN,6,NaN
3,1892-09-04,USC00087205,TMAX,322,NaN,NaN,6,NaN
4,1892-09-05,USC00087205,TMAX,333,NaN,NaN,6,NaN


In [ ]:
df = df[df["ELEMENT"].isin(["TMAX", "TMIN"])].copy()
df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")
df = df.dropna(subset=["DATE"])
df["TEMP_C"] = df["DATA_VALUE"] / 10.0  # tenths degC -> degC
df["DOY"] = df["DATE"].dt.dayofyear
df["YEAR"] = df["DATE"].dt.year

In [12]:
df.head()

,DATE,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,TEMP_C,DOY,YEAR
0,1892-09-01,USC00087205,TMAX,322,NaN,NaN,6,NaN,32.2,245,1892
1,1892-09-02,USC00087205,TMAX,317,NaN,NaN,6,NaN,31.7,246,1892
2,1892-09-03,USC00087205,TMAX,317,NaN,NaN,6,NaN,31.7,247,1892
3,1892-09-04,USC00087205,TMAX,322,NaN,NaN,6,NaN,32.2,248,1892
4,1892-09-05,USC00087205,TMAX,333,NaN,NaN,6,NaN,33.3,249,1892


In [13]:
tmin = df[df["ELEMENT"] == "TMIN"].copy()
tmin["temp_F"] = tmin["TEMP_C"] * 9/5 + 32
tmin["month"] = tmin["DATE"].dt.month
tmin["year"] = tmin["DATE"].dt.year

tmin_period = tmin[(tmin["year"] >= 1991) & (tmin["year"] <= 2020)].copy()

In [15]:
tmin_period["frost"]  = tmin_period["temp_F"] <= 32
tmin_period["freeze"] = tmin_period["temp_F"] <= 28

monthly_counts = (tmin_period.groupby(["year", "month"])[["frost", "freeze"]].sum())

mean_risk = monthly_counts.groupby("month").mean()
mean_risk

,frost,freeze
month,,
1,1.866667,0.500000
2,0.633333,0.066667
3,0.100000,0.000000
4,0.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000
8,0.000000,0.000000
9,0.033333,0.033333


October and November are basically frost-free...almost no days drop below freezing, and none get cold enough to damage plants.

Things start to cool down in December, with about half a day per month that reaches 28 or colder, and a little over half a day that hits 32 or colder. The real cold shows up in January, which has the highest risk: on average, almost two frost days per month and about half a freeze day.

So overall, strawberry plants are pretty safe early in the season, but December and especially January are the months where damaging cold becomes a realistic concern.

### Part 2 (25 points)

In [30]:
enso = pd.read_csv("sstoi.indices.txt", 
                   delim_whitespace=True,
                   header=0,
                   names=[
                       "year","month",
                       "NINO12","NINO12_anom",
                       "NINO3","NINO3_anom",
                       "NINO4","NINO4_anom",
                       "NINO34","NINO34_anom"
                   ])
enso

C:\Users\aliso\AppData\Local\Temp\ipykernel_2188\3653693203.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  enso = pd.read_csv("sstoi.indices.txt",


,year,month,NINO12,NINO12_anom,NINO3,NINO3_anom,NINO4,NINO4_anom,NINO34,NINO34_anom
0,1982,1,24.28,-0.24,25.84,0.17,28.01,-0.21,26.65,0.08
1,1982,2,25.38,-0.72,26.26,-0.11,27.99,-0.11,26.54,-0.20
2,1982,3,25.22,-1.38,26.92,-0.25,28.18,-0.05,27.09,-0.14
3,1982,4,24.57,-1.16,27.52,-0.05,28.61,0.10,27.83,0.02
4,1982,5,24.00,-0.62,27.70,0.49,29.19,0.40,28.37,0.49
...,...,...,...,...,...,...,...,...,...,...
522,2025,7,22.29,0.46,25.92,0.04,28.84,0.05,27.24,-0.06
523,2025,8,21.09,0.23,24.97,-0.24,28.63,-0.06,26.58,-0.33
524,2025,9,20.40,-0.18,24.60,-0.41,28.41,-0.27,26.32,-0.44
525,2025,10,20.83,-0.04,24.74,-0.35,28.36,-0.33,26.29,-0.48


In [31]:
freeze_monthly = (tmin_period.groupby(["year", "month"])["freeze"].sum().reset_index())
freeze_monthly

,year,month,freeze
0,1991,1,0
1,1991,2,0
2,1991,3,0
3,1991,4,0
4,1991,5,0
...,...,...,...
352,2020,8,0
353,2020,9,0
354,2020,10,0
355,2020,11,0


In [32]:
df_enso = pd.merge(freeze_monthly, enso, on=["year", "month"], how="inner")
df_enso

,year,month,freeze,NINO12,NINO12_anom,NINO3,NINO3_anom,NINO4,NINO4_anom,NINO34,NINO34_anom
0,1991,1,0,23.73,-0.78,25.63,-0.05,28.62,0.40,26.89,0.33
1,1991,2,0,25.70,-0.40,26.28,-0.10,28.40,0.30,26.87,0.14
2,1991,3,0,26.32,-0.28,26.96,-0.20,28.38,0.15,27.16,-0.08
3,1991,4,0,25.21,-0.52,27.39,-0.19,28.73,0.22,27.89,0.08
4,1991,5,0,24.65,0.04,27.44,0.22,29.18,0.39,28.13,0.25
...,...,...,...,...,...,...,...,...,...,...,...
352,2020,8,0,20.17,-0.69,24.79,-0.43,28.36,-0.34,26.38,-0.52
353,2020,9,0,19.82,-0.76,24.21,-0.80,28.25,-0.43,26.12,-0.64
354,2020,10,0,20.12,-0.76,24.19,-0.90,27.97,-0.71,25.64,-1.13
355,2020,11,0,20.95,-0.67,24.17,-1.03,27.98,-0.70,25.59,-1.23


In [ ]:
from scipy.stats import pearsonr

indices = ["NINO12_anom", "NINO3_anom", "NINO4_anom", "NINO34_anom"]

results = []

for idx in indices:
    valid = df_enso[["freeze", idx]].dropna()
    r, p = pearsonr(valid["freeze"], valid[idx])
    print(f"{idx}: r = {r:.3f}, p = {p:.3f}")
    results.append((idx, r, p))

best = max(results, key=lambda x: abs(x[1]))
print("\nBEST INDEX:", best[0], "with r =", round(best[1], 3), "and p =", round(best[2], 3))

NINO12_anom: r = -0.085, p = 0.107
NINO3_anom: r = -0.089, p = 0.094
NINO4_anom: r = -0.080, p = 0.131
NINO34_anom: r = -0.075, p = 0.158

BEST INDEX: NINO3_anom with r = -0.089 and p = 0.094


I computed monthly counts of freeze events for 1991–2020 and compared them with ENSO SST anomalies from the Nino1+2, Nino3, Nino4, and Nino3.4 regions using Pearson correlation. All correlations were weak, but the Nino 3 anomaly had the largest absolute correlation coefficient (r = -0.089).

This means that out of the four ENSO regions, the Nino 3 region is the one most closely related to freeze events in central Florida, although the overall relationship is very weak. The negative sign suggests that colder Nino 3 SSTs slightly increase freeze risk.